Purpose:
Retrieve full list of UK stations from http://orr.gov.uk/statistics/published-stats/station-usage-estimates

Create tables in the database and add data

Programmer:  Robin Linacre

In [1]:
import pandas as pd
pd.options.display.max_columns = 999
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

/Users/robinlinacre/anaconda/lib/python3.6/site-packages/sqlalchemy/dialects/postgresql/base.py:2526: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


In [2]:
sql = """
DROP TABLE IF EXISTS tt_h.all_stations;
DROP TABLE IF EXISTS tt_h.include_exclude;
"""
cursor.execute(sql)
conn.commit()

In [3]:
stations_df = pd.read_excel("http://www.orr.gov.uk/__data/assets/excel_doc/0020/23357/estimates-of-station-usage-2015-16.xlsx", 2)

In [4]:
headers = [h.lower() for h in list(stations_df.columns)]
headers = [h.replace(" ", "_").replace("(", "").replace(")","") for h in headers]
stations_df.columns = headers
stations_df.head()
stations_df = stations_df[pd.notnull(stations_df["nlc"])]

In [5]:
stations_df["nlc"] = stations_df["nlc"].astype(int)

In [6]:
# Now write out to postgres

stations_df.to_sql("all_stations", engine, schema="tt_h", if_exists="replace", index=False)

In [7]:
# Create geometry column for the points including a spatial index for efficient querying

sql = """
SELECT AddGeometryColumn ('tt_h', 'all_stations', 'geom', 27700, 'POINT', 2);
UPDATE tt_h.all_stations SET geom = ST_GeomFromText('POINT(' || os_grid_easting || ' ' || os_grid_northing || ')', 27700 );
CREATE INDEX idx_geom_all_stations_points ON tt_h.all_stations USING gist(geom);
"""
cursor.execute(sql)
conn.commit()

In [8]:
#Make a lat and lng column

sql = """
ALTER TABLE tt_h.all_stations ADD lat float, ADD lng float, ADD icscode text, 
ADD icscode_status text, ADD tfl_request text, ADD tfl_response json, ADD tfl_message  text;
UPDATE tt_h.all_stations SET
    lng = ST_X(ST_TRANSFORM(geom, 4326)),
    lat = ST_Y(ST_TRANSFORM(geom,4326));
ALTER TABLE tt_h.all_stations  ADD PRIMARY KEY (nlc);
""" 
cursor.execute(sql)
conn.commit()

In [9]:
%%bash
shp2pgsql -I -s 27700 shapefiles/include_exclude.shp tt_h.include_exclude | psql -d postgres

SET
SET
BEGIN
CREATE TABLE
ALTER TABLE
                       addgeometrycolumn                        
----------------------------------------------------------------
 tt_h.include_exclude.geom SRID:27700 TYPE:MULTIPOLYGON DIMS:2 
(1 row)

INSERT 0 1
CREATE INDEX
COMMIT
ANALYZE


Shapefile type: Polygon
Postgis type: MULTIPOLYGON[2]


In [10]:
sql = """
(SELECT s.nlc
from tt_h.all_stations as s, tt_h.include_exclude as ie
where ST_Contains(ie.geom, s.geom))
"""
nlcs = pd.read_sql(sql, conn)
nlcs = list(nlcs.loc[:, "nlc"])

In [11]:
sql = """
ALTER TABLE tt_h.all_stations ADD in_london boolean;
UPDATE tt_h.all_stations SET in_london = null;
"""
cursor.execute(sql)
conn.commit()

In [12]:
all_stations = Automapped_Base.classes.all_stations
stations = session.query(all_stations)
for station in stations:
    station.in_london = (station.nlc in nlcs)
    session.add(station)
session.commit()